## Loading libraries and data

Here we load the necessary libraries. Note that while `pandas`, `numpy`, `sklearn`, `statsmodels` come with Anaconda distribution, you will probably have to install `stargazer` (for printing regression tables) and `plotly` (data visualization) manually.

In [1]:
#@title
# This is to install some packages
!pip install stargazer
!pip install plotly
# You can comment it out later

In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf

from stargazer.stargazer import Stargazer, LineLocation
import plotly.express as px

# Notebook 2: logistic regression

Go through the notebook and do exercises. There will be topics for discussion in the end - we will discuss them in the next class.

By the end of this activity, you should

1. Learn how to train and evaluate a logistic regression model

2. Explain how to recognize overfitting

3. Explain importance of variable selection

The following command imports the CSV dataset using pandas:

In [3]:
#### Loading data from Google Drive - thanks to ChatGPT
# https://drive.google.com/file/d/1krC17OblVQ9tsEshKmv2hGBAeqJlqn83/view?usp=sharing

import requests
from io import StringIO

# Set the file ID of the CSV file you want to load
file_id = "1krC17OblVQ9tsEshKmv2hGBAeqJlqn83"

# Set the URL to download the file using the Drive API
url = f"https://drive.google.com/uc?id={file_id}&export=download"

# Make a GET request to download the file and decode the content
content = requests.get(url).content.decode("utf-8")

# Convert the string content to a pandas dataframe
dataset = pd.read_csv(StringIO(content))
dataset.shape

(1428, 501)

Here is what the data look like. Note that "d_k" here is the bitcoin price $k$ days ago for $k=0,1,2,\dots,500$.

In [4]:
dataset.head()

,d_500,d_499,d_498,d_497,d_496,d_495,d_494,d_493,d_492,d_491,...,d_9,d_8,d_7,d_6,d_5,d_4,d_3,d_2,d_1,d_0
0,457.334015,424.440002,394.795990,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,...,410.261993,382.492004,387.490997,402.971008,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005
1,424.440002,394.795990,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,...,382.492004,387.490997,402.971008,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998
2,394.795990,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,377.181000,...,387.490997,402.971008,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998,373.056000
3,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,377.181000,375.467010,...,402.971008,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998,373.056000,374.447998
4,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,377.181000,375.467010,386.944000,...,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998,373.056000,374.447998,369.949005


And some simple description

In [5]:
dataset.describe()

,d_500,d_499,d_498,d_497,d_496,d_495,d_494,d_493,d_492,d_491,...,d_9,d_8,d_7,d_6,d_5,d_4,d_3,d_2,d_1,d_0
count,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,...,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000,1428.000000
mean,2648.691245,2652.788714,2656.927572,2661.259389,2665.471473,2669.748257,2673.884376,2678.123150,2682.292271,2686.580300,...,4923.055434,4927.812144,4932.599486,4937.363959,4942.341983,4947.218665,4952.071872,4956.889928,4961.692923,4966.532289
std,3781.067942,3781.861871,3782.655483,3783.604946,3784.443677,3785.331134,3786.078451,3786.959250,3787.752106,3788.645599,...,3951.121074,3949.772892,3948.402616,3947.018959,3945.778664,3944.453712,3943.108585,3941.739581,3940.337489,3938.949273
min,178.102997,178.102997,178.102997,178.102997,178.102997,178.102997,178.102997,178.102997,178.102997,178.102997,...,368.766998,368.766998,368.766998,368.766998,368.766998,368.766998,368.766998,368.766998,368.766998,368.766998
25%,350.762497,350.762497,350.762497,350.762497,350.762497,350.762497,350.762497,350.762497,350.762497,350.762497,...,915.989243,919.018479,919.686493,920.224014,920.854523,921.445526,921.739258,921.935257,924.000733,931.066742
50%,637.373016,638.303009,638.919494,639.541504,640.134003,640.470001,640.817016,641.351501,644.315979,647.329987,...,4141.202392,4147.023438,4154.851562,4159.401611,4160.945069,4162.169922,4168.899902,4178.330078,4187.815185,4197.185059
75%,3895.750061,3910.704956,3925.244995,3944.987549,4017.604980,4067.214966,4076.859986,4090.874939,4106.125000,4124.907471,...,7842.702515,7842.702515,7842.702515,7842.702515,7842.702515,7842.702515,7842.702515,7842.702515,7842.702515,7842.702515
max,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,...,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391,19497.400391


## Data exploration

The variable "d_0" is the price of bitcoin at a given day. Below is the time series plot.

In [6]:
fig = px.line(dataset,  y = 'd_0')
fig.update_layout(xaxis_title='Day', yaxis_title = "Bitcoin price")
fig.show()

## Data processing

This is an exercise on classification, i.e., we will be interested in whether the price of bitcoin goes up or down. We will add a new variable called "move". Its value will be 1 if the bitcoin price goes up and 0 if it goes down or stays the same.

In [7]:
dataset["move"] = 0.0 + (dataset['d_0'] > dataset['d_1'])
# Explanation: (dataset['d_0'] > dataset['d_1']) is a logical vector.
# It has TRUE at positions where d_0 > d_1 and FALSE at positions where d_0 <= d_1
# When a logical value is converted to numeric, TRUE becomes 1 and FALSE becomes 0.
# "0.0 + " is needed to convert logical to numeric

dataset

,d_500,d_499,d_498,d_497,d_496,d_495,d_494,d_493,d_492,d_491,...,d_8,d_7,d_6,d_5,d_4,d_3,d_2,d_1,d_0,move
0,457.334015,424.440002,394.795990,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,...,382.492004,387.490997,402.971008,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,0.0
1,424.440002,394.795990,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,...,387.490997,402.971008,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998,0.0
2,394.795990,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,377.181000,...,402.971008,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998,373.056000,1.0
3,408.903992,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,377.181000,375.467010,...,391.726013,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998,373.056000,374.447998,1.0
4,398.821014,402.152008,435.790985,423.204987,411.574005,404.424988,399.519989,377.181000,375.467010,386.944000,...,392.153015,394.971985,380.289001,379.473999,378.255005,368.766998,373.056000,374.447998,369.949005,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,6184.709961,6295.729980,6322.689941,6297.569824,6199.709961,6308.520020,6334.729980,6580.629883,6423.759766,6506.069824,...,7152.301758,6932.480469,6640.515137,7276.802734,7202.844238,7218.816406,7191.158691,7511.588867,7355.628418,0.0
1424,6295.729980,6322.689941,6297.569824,6199.709961,6308.520020,6334.729980,6580.629883,6423.759766,6506.069824,6308.529785,...,6932.480469,6640.515137,7276.802734,7202.844238,7218.816406,7191.158691,7511.588867,7355.628418,7322.532227,0.0
1425,6322.689941,6297.569824,6199.709961,6308.520020,6334.729980,6580.629883,6423.759766,6506.069824,6308.529785,6488.759766,...,6640.515137,7276.802734,7202.844238,7218.816406,7191.158691,7511.588867,7355.628418,7322.532227,7275.155762,0.0
1426,6297.569824,6199.709961,6308.520020,6334.729980,6580.629883,6423.759766,6506.069824,6308.529785,6488.759766,6376.709961,...,7276.802734,7202.844238,7218.816406,7191.158691,7511.588867,7355.628418,7322.532227,7275.155762,7238.966797,0.0


## Training and test datasets

Here we will split the entire dataset into 70% training and 30% test data. We will also remove the original variable "d_0" (think why).

In [8]:
train_data, test_data = train_test_split(dataset, test_size=0.3, random_state = 8128)
train_data = train_data.drop('d_0', axis = 1)
test_data = test_data.drop('d_0', axis = 1)
print("Entire dataset dimensions =", dataset.shape)
print("Train data dimensions =", train_data.shape)
print("Test data dimensions =", test_data.shape)

Entire dataset dimensions = (1428, 502)
Train data dimensions = (999, 501)
Test data dimensions = (429, 501)


### Exercise 1

Does the bitcoin price more often go up or down in the training data? Imagine that instead of training a logistic regression, we will use a super-simple baseline model - always predict that the price goes up or down (whichever is more common in the training data). What will be the test accuracy of such a baseline model?

In [9]:
### Write your code here

fraction_up_days = np.mean(train_data['move'])
print("Fraction of days on which the price goes up in the training data is", np.round(fraction_up_days, 2))
baseline_prediction = 0. # Enter the correct code here
print("The baseline prediction is", baseline_prediction)
baseline_test_acc = np.mean(0. + (test_data['move'] == baseline_prediction))
print("Test accuracy of baseline model =", baseline_test_acc)

Fraction of days on which the price goes up in the training data is 0.57
The baseline prediction is 0.0
Test accuracy of baseline model = 0.47785547785547783


## Modelling

Here we train two logistic models. The first one uses 5 historical prices as predictors and the second one uses 300 historical prices as predictors. For the second regression, we will create a formula in a loop (let Fedor know if there is a better method) as follows:

In [10]:
f = "move ~" + " d_1"
for i in range(2, 301):
    f = f + " + d_" + str(i)

print(f[0:100])

move ~ d_1 + d_2 + d_3 + d_4 + d_5 + d_6 + d_7 + d_8 + d_9 + d_10 + d_11 + d_12 + d_13 + d_14 + d_15


In [11]:
mod1 = smf.logit(formula = 'move ~ d_1 + d_2 + d_3 + d_4 + d_5', data = train_data).fit()
mod2 = smf.logit(formula = f, data = train_data).fit()

stargazer = Stargazer([mod1])
# We will not report coefficients of the second model since there are too many of them
stargazer

Optimization terminated successfully.
         Current function value: 0.673967
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.426035
         Iterations 10


### Model evaluation

To evaluate our model, we need some performance metric. Such a performance metric can either show how good a model is (accuracy) or how bad a model is (error or loss function). For classification models, we usually use metrics related to accuracy (accuracy, balanced accuracy, precision, recall, F1-score etc). Here we will just look at the overall accuracy, i.e.,
$$
\frac{\mbox{Number of correctly predicted observations}}{\mbox{Total number of observations}}
$$

Note that the accuracy is not a differentiable function and hence its impossible to use it to train logistic regression by gradient descent. To train logistic regression, we use the binary cross-entropy loss
$$
L(\beta)=-\sum_{i=1}^{N}\left(y^i\log p(x^i)+ (1-y^i)\log(1-p(x^i))\right)
$$
It means that, by definition, model parameters are found as to minimize it, i.e.,
$$
(\beta_0,\beta_1,\dots,\beta_p)=\arg\min L
$$
However, the binary cross-entropy loss is not interpretable and it doesn't make much sense to use it to evaluate the final model.

First, we will construct predictions of model 1 (with 5 predictors).

In [12]:
mod1.predict(test_data)

1087    0.590177
280     0.637140
809     0.564082
1120    0.594818
28      0.639604
          ...   
194     0.636629
795     0.524401
817     0.495739
204     0.637022
496     0.618517
Length: 429, dtype: float64

Note that logistic regression reports probabilities rather than labels. This is actually a good thing because not only it tells us which label it predicts, it also report how confident it is in its predictions. We will just use the standard convention that if the probability is above 0.5, then the predicted label is 1, otherwise it is 0. Here are some predicted labels.

In [13]:
0. + (mod1.predict(test_data) > 0.5)

1087    1.0
280     1.0
809     1.0
1120    1.0
28      1.0
       ... 
194     1.0
795     1.0
817     0.0
204     1.0
496     1.0
Length: 429, dtype: float64

Now we are ready to construct predictions and report training and test accuracies of the two models

In [14]:
pred1_train = 0. + (mod1.predict(train_data) > 0.5)
pred1_test = 0. + (mod1.predict(test_data) > 0.5)

# Note that since we trained the 500-regressor model in a different manner (without the formula API),
# we need to use the predict method in a different way too:
pred2_train = 0. + (mod2.predict(train_data) > 0.5)
pred2_test = 0. + (mod2.predict(test_data) > 0.5)

print("Training accuracy of model 1 =", metrics.accuracy_score(train_data["move"], pred1_train))
print("Test accuracy of model 1 =", metrics.accuracy_score(test_data["move"], pred1_test))
print("Training accuracy of model 2 =", metrics.accuracy_score(train_data["move"], pred2_train))
print("Test accuracy of model 2 =", metrics.accuracy_score(test_data["move"], pred2_test))

Training accuracy of model 1 = 0.5795795795795796
Test accuracy of model 1 = 0.5244755244755245
Training accuracy of model 2 = 0.7857857857857858
Test accuracy of model 2 = 0.48951048951048953


### Exercise 2

Write a Python function with three inputs - a logistic regression model, a dataset, and the name of the column containing the response variable. The output of such a function should be the overall accuracy of the regression model. You will need to modify the following code from Notebook 1 to do that:

In [15]:
# Modify this code
# Change the name of the function "model_mae" to "model_acc" and do other necessary changes

def model_acc(model, data = test_data, y = 'move'):
    pred = 0. * (model.predict(data) > 0.5) # change this code to a correct one
    return metrics.accuracy_score(data[y], pred)

print("Test accuracy of model 1 =", model_acc(mod1))
print("Test accuracy of model 2 =", model_acc(mod2))

Test accuracy of model 1 = 0.47785547785547783
Test accuracy of model 2 = 0.47785547785547783


## Exercise 3

Look at the baseline model again. Is the logistic regression better than the baseline model? What if we use fewer predictors or more predictors?

In [16]:
print("Test accuracy of model 1 =", metrics.accuracy_score(test_data["move"], pred1_test))
print("Test accuracy of baseline model =", baseline_test_acc)

Test accuracy of model 1 = 0.5244755244755245
Test accuracy of baseline model = 0.47785547785547783


_WRITE YOUR TEXT HERE_

In [17]:
mod3 = smf.logit(formula = 'move ~ d_1', data = train_data).fit() # EXPERIMENT WITH THIS FORMULA
print("Test accuracy of model 3 =", model_acc(mod3))

Optimization terminated successfully.
         Current function value: 0.675360
         Iterations 4
Test accuracy of model 3 = 0.47785547785547783


_WRITE YOUR TEXT HERE_

## Question to think about

Think about these questions at home - we will discuss them in next class

### Question 1 - super-complex model

Why do you think does the second model appear to be so much more accurate on the training data but performs so poorly on the test data? What do we do to prevent such a situation?

### Question 2 - variable selection

Our experiments show that we can try models with a different number of predictors and it is important to choose the number of predictors wisely. What do you think of the following way to do it?

We will train logistic regressions $f(X_1), f(X_1,X_2), f(X_1,X_2,X_3),\ldots, f(X_1,X_2,\dots,X_{500})$ (where $X_d$ is the bitcoin price $d$ days ago). Every one of them will be trained on the train data and but we will use the test accuracy to evaluate its performance. Then we will choose the model with the highest test accuracy.

Is it a correct way to choose the right number of predictors?

# Theoretical homework

### Problem 1

A linear regression is usually trained by minimizing the mean squared error loss
$$
L(\beta)=\frac{1}{N}\sum_{i=1}^{N}\left(y^i-\beta_0-
\sum_{j=1}^{p}\beta_j x^i_j\right)^2
$$
The reason is that the first order conditions $\frac{\partial L}{\partial \beta_i}=0$ for $i=0,1,\dots,p$ can be solved in closed form (you learned how to do it in regression analysis). However, it is easier to interpret the mean absolute error, i.e.,
$$
\frac{1}{N}\sum_{i=1}^{N}\left|y^i-\beta_0-
\sum_{j=1}^{p}\beta_j x^i_j\right|
$$
Compute partial derivatives of the mean absolute error with respect to model parameters. Is the mean absolute error continuous? differentiable? If not, is non-continuity or non-differentiability going to be a problem for training the model by gradient descent?

### Answer



### Problem 2

Recall from the lecture that the loss function for logistic regression is the binary cross-entropy
$$
L(\beta)=-\sum_{i=1}^{N}\left(y^i\log p(x^i)+ (1-y^i)\log(1-p(x^i))\right)
$$
Compute its gradient, i.e., find partial derivatives
$$
\frac{\partial L}{\partial \beta_i}
$$
for $i=0,1,\dots,p$.

### Answer:



### Problem 3

Explain why adding more predictors to a linear model will always reduce the training mean squared error. Is it true that adding more predictors to the logistic model will always increase the training accuracy? Either prove it or provide a counter-example.

### Answer

